# GitHub Trending RSS Feed Generator

Questo notebook crea automaticamente un feed RSS delle repository più popolari su GitHub utilizzando l'API di GitHub e Python.

## Importa le Librerie Necessarie

In [1]:
# Importa le librerie necessarie
import requests
import json
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import os
from dateutil import parser
import time

## Autenticazione con l'API di GitHub

Per utilizzare l'API di GitHub è necessario un token personale per evitare limiti di richieste.
Puoi creare un token personale su: https://github.com/settings/tokens

In [2]:
# Configurazione del token di autenticazione GitHub
# Utilizza una variabile d'ambiente o inserisci direttamente il token
# (Ma è meglio non includere token direttamente nel codice)

# Ottieni il token da una variabile d'ambiente
GITHUB_TOKEN = os.getenv('GITHUB_TOKEN')
    
# Configura gli headers per le richieste API
headers = {
    'Authorization': f'token {GITHUB_TOKEN}',
    'Accept': 'application/vnd.github.v3+json'
}

# Verifica se il token è stato impostato
if not GITHUB_TOKEN:
    print("Attenzione: Token GitHub non trovato! Le richieste API potrebbero essere limitate.")
    headers = {'Accept': 'application/vnd.github.v3+json'}
else:
    print("Token GitHub trovato. Le richieste API saranno autenticate.")

Token GitHub trovato. Le richieste API saranno autenticate.


## Recupera le Top Repository

Utilizziamo l'API di GitHub per recuperare le repository più popolari basate su criteri come stelle, lingua e data di creazione.

In [5]:
def get_trending_repositories(language=None, since='daily', limit=20):
    """
    Recupera le repository di tendenza su GitHub
    
    Parameters:
    language (str): Linguaggio di programmazione da filtrare (opzionale)
    since (str): Periodo di tempo ('daily', 'weekly', 'monthly')
    limit (int): Numero massimo di repository da recuperare
    
    Returns:
    list: Lista delle repository più popolari
    """
    # L'API Search di GitHub è più efficace per questa operazione
    query = "stars:>50"
    
    if language:
        query += f" language:{language}"
        
    # Ordina per stelle in ordine decrescente
    sort = "stars"
    order = "desc"
    
    # Determina il periodo di tempo
    if since == 'weekly':
        query += " created:>{}".format((datetime.now() - timedelta(days=7)).strftime("%Y-%m-%d"))
    elif since == 'monthly':
        query += " created:>{}".format((datetime.now() - timedelta(days=30)).strftime("%Y-%m-%d"))
    elif since == 'daily':
        query += " created:>{}".format((datetime.now() - timedelta(days=1)).strftime("%Y-%m-%d"))
        
    # Costruisci l'URL dell'API
    url = f"https://api.github.com/search/repositories?q={query}&sort={sort}&order={order}&per_page={limit}"
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()  # Solleva un'eccezione per HTTP error
        
        data = response.json()
        return data.get('items', [])
    except requests.exceptions.RequestException as e:
        print(f"Errore durante la richiesta API: {e}")
        return []

# Testa la funzione
trending_repos = get_trending_repositories(since='monthly', limit=100)
print(f"Recuperate {len(trending_repos)} repository di tendenza")

Recuperate 100 repository di tendenza


## Filtra e Organizza i Dati Essenziali

Estrai le informazioni rilevanti dalle repository recuperate e organizzale in una struttura dati appropriata.

In [21]:
def extract_repo_data(repos):
    """
    Estrae i dati essenziali dalle repository
    
    Parameters:
    repos (list): Lista di repository dall'API GitHub
    
    Returns:
    list: Lista di dizionari con i dati organizzati
    """
    organized_data = []
    
    for repo in repos:
        # Estrai i dati essenziali
        repo_data = {
            'name': repo.get('full_name', ''),
            'url': repo.get('html_url', ''),
            'description': repo.get('description', 'no description'),
            'stars': repo.get('stargazers_count', 0),
            'forks': repo.get('forks_count', 0),
            'language': repo.get('language', 'no language'),
            'created_at': repo.get('created_at'),
            'updated_at': repo.get('updated_at'),
            'owner': {
                'name': repo.get('owner', {}).get('login', ''),
                'avatar_url': repo.get('owner', {}).get('avatar_url', '')
            }
        }
        
        organized_data.append(repo_data)
    
    return organized_data

# Testa la funzione
organized_repos = extract_repo_data(trending_repos)

# Mostra un esempio dei dati organizzati
if organized_repos:
    print("Esempio di repository organizzata:")
    print(json.dumps(organized_repos[0], indent=2))
else:
    print("Nessuna repository disponibile")

Esempio di repository organizzata:
{
  "name": "riseandignite/mcp-shield",
  "url": "https://github.com/riseandignite/mcp-shield",
  "description": "Security scanner for MCP servers",
  "stars": 195,
  "forks": 5,
  "language": "TypeScript",
  "created_at": "2025-04-15T04:07:44Z",
  "updated_at": "2025-04-15T21:42:50Z",
  "owner": {
    "name": "riseandignite",
    "avatar_url": "https://avatars.githubusercontent.com/u/190351315?v=4"
  }
}


## Genera il Feed RSS

Crea un feed RSS XML utilizzando i dati delle repository estratti.

In [22]:
def create_rss_feed(repos, title="GitHub Trending Repositories", description="Le repository più popolari su GitHub"):
    """
    Crea un feed RSS dalle repository di GitHub
    
    Parameters:
    repos (list): Lista di repository organizzate
    title (str): Titolo del feed RSS
    description (str): Descrizione del feed RSS
    
    Returns:
    str: Feed RSS in formato XML
    """
    # Crea l'elemento root del feed RSS
    rss = ET.Element("rss", version="2.0")
    channel = ET.SubElement(rss, "channel")
    
    # Aggiungi i dettagli del feed
    ET.SubElement(channel, "title").text = title
    ET.SubElement(channel, "link").text = "https://github.com/trending"
    ET.SubElement(channel, "description").text = description
    ET.SubElement(channel, "language").text = "it-IT"
    ET.SubElement(channel, "pubDate").text = datetime.now().strftime("%a, %d %b %Y %H:%M:%S +0000")
    
    # Aggiungi ogni repository come item nel feed
    for repo in repos:
        item = ET.SubElement(channel, "item")
        
        # Formatta il titolo con il nome e le stelle
        item_title = f"{repo['name']} ({repo['stars']} ⭐)"
        ET.SubElement(item, "title").text = item_title
        
        ET.SubElement(item, "link").text = repo['url']
        
        # Formatta la descrizione HTML
        description = f"""
        <p><strong>Repository:</strong> {repo['name']}</p>
        <p><strong>Descrizione:</strong> {repo['description']}</p>
        <p><strong>Linguaggio:</strong> {repo['language']}</p>
        <p><strong>Stelle:</strong> {repo['stars']} ⭐</p>
        <p><strong>Fork:</strong> {repo['forks']}</p>
        <p><strong>Creata il:</strong> {repo['created_at']}</p>
        <p><a href="{repo['url']}">Visita repository</a></p>
        """
        ET.SubElement(item, "description").text = description
        
        # Genera un ID unico
        ET.SubElement(item, "guid", isPermaLink="false").text = repo['url']
        
        # Aggiungi data di pubblicazione
        if repo.get('created_at'):
            try:
                pub_date = parser.parse(repo['created_at']).strftime("%a, %d %b %Y %H:%M:%S +0000")
                ET.SubElement(item, "pubDate").text = pub_date
            except Exception:
                ET.SubElement(item, "pubDate").text = datetime.now().strftime("%a, %d %b %Y %H:%M:%S +0000")
    
    # Converti l'albero XML in stringa
    xml_string = ET.tostring(rss, encoding='utf-8', method='xml')
    
    # Aggiungi l'intestazione XML e decodifica
    xml_declaration = '<?xml version="1.0" encoding="UTF-8" ?>\n'
    return xml_declaration + xml_string.decode('utf-8')

In [23]:
# Testa la generazione del feed RSS
if organized_repos:
    rss_feed = create_rss_feed(organized_repos)
    print("Anteprima del feed RSS generato:")
    print(rss_feed[:500] + "...")  # Mostra solo i primi 500 caratteri
else:
    print("Nessuna repository disponibile per generare il feed RSS")

Anteprima del feed RSS generato:
<?xml version="1.0" encoding="UTF-8" ?>
<rss version="2.0"><channel><title>GitHub Trending Repositories</title><link>https://github.com/trending</link><description>Le repository più popolari su GitHub</description><language>it-IT</language><pubDate>Tue, 15 Apr 2025 23:59:07 +0000</pubDate><item><title>riseandignite/mcp-shield (195 ⭐)</title><link>https://github.com/riseandignite/mcp-shield</link><description>
        &lt;p&gt;&lt;strong&gt;Repository:&lt;/strong&gt; riseandignite/mcp-shield&lt;/...


## Salva il Feed RSS su File

Salva il feed RSS generato in un file XML per la distribuzione.

In [24]:
def save_rss_feed(rss_feed, filename="github_trending.xml"):
    """
    Salva il feed RSS in un file
    
    Parameters:
    rss_feed (str): Contenuto del feed RSS
    filename (str): Nome del file di output
    
    Returns:
    bool: True se il salvataggio ha avuto successo, False altrimenti
    """
    try:
        with open(filename, "w", encoding="utf-8") as file:
            file.write(rss_feed)
        return True
    except Exception as e:
        print(f"Errore durante il salvataggio del file: {e}")
        return False

# Salva il feed RSS in un file
if 'rss_feed' in locals():
    success = save_rss_feed(rss_feed)
    if success:
        print(f"Il feed RSS è stato salvato in 'github_trending.xml'")
    else:
        print("Si è verificato un errore durante il salvataggio del feed RSS")

Il feed RSS è stato salvato in 'github_trending.xml'


## Esecuzione Completa

Esegui il processo completo per generare e salvare il feed RSS.

In [25]:
def main():
    """
    Processo completo per generare e salvare il feed RSS
    """
    print("Avvio generazione feed RSS dalle repository GitHub trending...")
    
    # 1. Recupera le repository di tendenza
    print("Recupero delle repository di tendenza...")
    repos = get_trending_repositories(limit=30)
    
    if not repos:
        print("Nessuna repository trovata. Verifica la connessione o i limiti dell'API.")
        return
    
    print(f"Recuperate {len(repos)} repository")
    
    # 2. Filtra e organizza i dati
    print("Elaborazione dei dati...")
    organized_repos = extract_repo_data(repos)
    
    # 3. Genera il feed RSS
    print("Generazione del feed RSS...")
    rss_feed = create_rss_feed(organized_repos)
    
    # 4. Salva il feed RSS
    print("Salvataggio del feed RSS...")
    success = save_rss_feed(rss_feed)
    
    if success:
        print("Feed RSS generato e salvato con successo!")
    else:
        print("Errore durante il salvataggio del feed RSS")

# Esegui il processo completo
if __name__ == "__main__":
    main()

Avvio generazione feed RSS dalle repository GitHub trending...
Recupero delle repository di tendenza...
Recuperate 2 repository
Elaborazione dei dati...
Generazione del feed RSS...
Salvataggio del feed RSS...
Feed RSS generato e salvato con successo!
Recuperate 2 repository
Elaborazione dei dati...
Generazione del feed RSS...
Salvataggio del feed RSS...
Feed RSS generato e salvato con successo!


## Conclusione

Questo notebook ha creato un feed RSS delle repository di tendenza su GitHub utilizzando l'API di GitHub. Il feed può essere distribuito tramite un server web o usato con servizi di aggregazione RSS.

Per migliorare ulteriormente:
- Implementare un meccanismo di aggiornamento automatico
- Aggiungere filtri per linguaggio, periodo di tempo e altre categorie
- Integrare con sistemi di notifica per nuove repository interessanti

## Generazione del Sito Web per i Feed RSS

In questa sezione, genereremo un sito web semplice che permetterà agli utenti di accedere ai feed RSS per diversi linguaggi di programmazione e periodi di tempo (giornaliero, settimanale, mensile).

In [32]:
def generate_website():
    """
    Genera un sito web HTML per visualizzare i feed RSS delle repository di tendenza
    organizzate per linguaggio di programmazione e periodo di tempo.
    """
    # Definisci i linguaggi supportati
    languages = [
        "All Languages", "Unknown languages", "1C Enterprise",
        "2-Dimensional Array", "4D", "ABAP", "ABAP CDS", "ABNF",
        "ActionScript", "Ada", "Adblock Filter List", "Adobe Font Metrics",
        "Agda", "AGS Script", "AIDL"
        # Puoi aggiungere altri linguaggi a piacimento
    ]
    
    # Definisci i periodi di tempo
    periods = ["Daily", "Weekly", "Monthly"]
    
    # Crea la struttura base del sito HTML
    html = """<!DOCTYPE html>
<html lang="it">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>GitHub Trending RSS</title>
    <style>
        body {
            font-family: -apple-system, BlinkMacSystemFont, 'Segoe UI', Roboto, 'Helvetica Neue', Arial, sans-serif;
            line-height: 1.6;
            color: #333;
            margin: 0;
            padding: 0;
            background-color: #f5f5f5;
        }
        header {
            background-color: #1a92ff;
            color: white;
            text-align: center;
            padding: 1.5rem 0;
            margin-bottom: 2rem;
        }
        h1 {
            margin: 0;
        }
        .build-info {
            font-size: 0.9rem;
            margin-top: 0.5rem;
        }
        .container {
            max-width: 1200px;
            margin: 0 auto;
            padding: 0 20px;
            display: flex;
            flex-wrap: wrap;
            justify-content: space-around;
        }
        .language-card {
            background: white;
            border-radius: 8px;
            box-shadow: 0 2px 5px rgba(0,0,0,0.1);
            width: 300px;
            margin-bottom: 30px;
            padding: 20px;
            text-align: center;
        }
        .language-title {
            font-size: 1.25rem;
            font-weight: 600;
            margin-bottom: 15px;
        }
        .feed-links {
            display: flex;
            justify-content: space-around;
        }
        a {
            color: #0366d6;
            text-decoration: none;
        }
        a:hover {
            text-decoration: underline;
        }
        .github-badge {
            display: inline-block;
            margin: 10px 0;
            padding: 5px 10px;
            background-color: #24292e;
            color: white;
            border-radius: 3px;
            font-size: 0.8rem;
        }
        .github-star {
            display: inline-block;
            margin: 10px 0;
            padding: 5px 10px;
            background-color: #fff;
            color: #24292e;
            border: 1px solid #e1e4e8;
            border-radius: 3px;
            font-size: 0.8rem;
        }
        footer {
            text-align: center;
            padding: 20px;
            margin-top: 20px;
            background-color: #f1f1f1;
            font-size: 0.9rem;
        }
    </style>
</head>
<body>
    <header>
        <h1>GitHub Trending RSS</h1>
        <div class="github-badge">Fetch and Generate RSS Feeds <span class="passing">passing</span></div>
        <div class="github-star">★ Star: 24</div>
        <div class="build-info">The latest build: """ + datetime.now().strftime("%d %B, %Y") + """</div>
    </header>
    
    <div class="container">
"""
    
    # Aggiungi una card per ogni linguaggio
    for language in languages:
        html += f"""
        <div class="language-card">
            <div class="language-title">{language}</div>
            <div class="feed-links">
"""
        # Aggiungi i link per ogni periodo di tempo
        for period in periods:
            language_param = language.replace(" ", "%20")
            language_filename = language.replace(" ", "_").lower()
            period_lower = period.lower()
            html += f"""
                <a href="feeds/{language_filename}_{period_lower}.xml">{period}</a>
"""
            
        html += """
            </div>
        </div>
"""
    
    # Chiudi l'HTML
    html += """
    </div>
    
    <footer>
        <p>Generato automaticamente con Python | <a href="https://github.com/duccioo/GitHubTrendingRSS">Codice sorgente</a></p>
    </footer>
</body>
</html>
"""
    
    return html

In [35]:
def save_website(html_content, filename="index.html"):
    """
    Salva il contenuto HTML in un file
    
    Parameters:
    html_content (str): Contenuto HTML del sito web
    filename (str): Nome del file di output
    
    Returns:
    bool: True se il salvataggio ha avuto successo, False altrimenti
    """
    try:
        with open(filename, "w", encoding="utf-8") as file:
            file.write(html_content)
        return True
    except Exception as e:
        print(f"Errore durante il salvataggio del file HTML: {e}")
        return False

# Genera e salva il sito web
html_content = generate_website()
success = save_website(html_content)

if success:
    print(f"Il sito web è stato salvato in 'index.html'")
else:
    print("Si è verificato un errore durante il salvataggio del sito web")

Il sito web è stato salvato in 'index.html'


In [34]:
def generate_all_feeds():
    """
    Genera feed RSS per tutti i linguaggi e periodi di tempo definiti
    e li salva nella cartella 'feeds'.
    """
    # Definisci i linguaggi supportati
    languages = [
        "All Languages", "Unknown languages"
        
        # Puoi aggiungere altri linguaggi a piacimento
    ]
    
    # Definisci i periodi di tempo
    periods = ["daily", "weekly", "monthly"]
    
    # Crea la cartella feeds se non esiste
    if not os.path.exists('feeds'):
        os.makedirs('feeds')
        print("Creata cartella 'feeds'")
    
    count = 0
    errors = 0
    
    # Genera feed per ogni combinazione di linguaggio e periodo
    for language in languages:
        for period in periods:
            try:
                print(f"Generazione feed per {language} ({period})...")
                
                # Usa None per 'All Languages' e converti appropriatamente per 'Unknown languages'
                lang_param = None if language == "All Languages" else language
                if language == "Unknown languages":
                    lang_param = "Unknown"
                
                # Recupera repository
                repos = get_trending_repositories(language=lang_param, since=period, limit=30)
                
                if repos:
                    # Organizza i dati
                    organized_repos = extract_repo_data(repos)
                    
                    # Genera feed RSS
                    feed_title = f"GitHub Trending - {language} ({period})"
                    feed_description = f"Le repository più popolari in {language} nell'ultimo periodo {period}"
                    rss_feed = create_rss_feed(organized_repos, title=feed_title, description=feed_description)
                    
                    # Salva feed RSS
                    filename = f"feeds/{language.replace(' ', '_').lower()}_{period}.xml"
                    with open(filename, 'w', encoding='utf-8') as file:
                        file.write(rss_feed)
                    
                    count += 1
                    print(f"Feed salvato: {filename}")
                    
                    # Aggiungi piccola pausa per evitare troppe richieste API
                    time.sleep(1)
                else:
                    print(f"Nessuna repository trovata per {language} ({period})")
                    errors += 1
                    
            except Exception as e:
                print(f"Errore per {language} ({period}): {str(e)}")
                errors += 1
    
    print(f"\nCompletato! Feed generati: {count}, errori: {errors}")
    return count, errors

# Esegui la generazione di tutti i feed
feeds_count, feeds_errors = generate_all_feeds()

Generazione feed per All Languages (daily)...
Nessuna repository trovata per All Languages (daily)
Generazione feed per All Languages (weekly)...
Nessuna repository trovata per All Languages (daily)
Generazione feed per All Languages (weekly)...
Feed salvato: feeds/all_languages_weekly.xml
Feed salvato: feeds/all_languages_weekly.xml
Generazione feed per All Languages (monthly)...
Generazione feed per All Languages (monthly)...
Feed salvato: feeds/all_languages_monthly.xml
Feed salvato: feeds/all_languages_monthly.xml
Generazione feed per Unknown languages (daily)...
Generazione feed per Unknown languages (daily)...
Nessuna repository trovata per Unknown languages (daily)
Generazione feed per Unknown languages (weekly)...
Nessuna repository trovata per Unknown languages (daily)
Generazione feed per Unknown languages (weekly)...
Feed salvato: feeds/unknown_languages_weekly.xml
Feed salvato: feeds/unknown_languages_weekly.xml
Generazione feed per Unknown languages (monthly)...
Generazion

## Esecuzione Finale

Eseguiamo tutte le funzioni principali in sequenza per generare il sito completo con tutti i feed RSS.

In [36]:
def main_complete():
    """
    Processo completo per generare tutti i feed RSS e il sito web
    """
    print("Avvio generazione del sistema completo di feed RSS GitHub trending...\n")
    
    # 1. Genera tutti i feed RSS
    print("\n=== GENERAZIONE FEED RSS ===\n")
    feeds_count, feeds_errors = generate_all_feeds()
    
    # 2. Genera il sito web
    print("\n=== GENERAZIONE SITO WEB ===\n")
    html_content = generate_website()
    success = save_website(html_content)
    
    if success:
        print(f"Il sito web è stato salvato con successo in 'index.html'")
    else:
        print(f"Si è verificato un errore durante il salvataggio del sito web")
    
    print("\n=== RIEPILOGO ===\n")
    print(f"Feed RSS generati: {feeds_count}")
    print(f"Feed RSS con errori: {feeds_errors}")
    print(f"Sito web generato: {'Sì' if success else 'No'}")
    print("\nPuoi aprire index.html nel tuo browser per visualizzare e accedere ai feed RSS")

# Eseguiamo tutto il processo completo se lo script viene eseguito direttamente
if __name__ == "__main__":
    main_complete()

Avvio generazione del sistema completo di feed RSS GitHub trending...


=== GENERAZIONE FEED RSS ===

Generazione feed per All Languages (daily)...
Nessuna repository trovata per All Languages (daily)
Generazione feed per All Languages (weekly)...
Nessuna repository trovata per All Languages (daily)
Generazione feed per All Languages (weekly)...
Feed salvato: feeds/all_languages_weekly.xml
Feed salvato: feeds/all_languages_weekly.xml
Generazione feed per All Languages (monthly)...
Generazione feed per All Languages (monthly)...
Feed salvato: feeds/all_languages_monthly.xml
Feed salvato: feeds/all_languages_monthly.xml
Generazione feed per Unknown languages (daily)...
Generazione feed per Unknown languages (daily)...
Nessuna repository trovata per Unknown languages (daily)
Generazione feed per Unknown languages (weekly)...
Nessuna repository trovata per Unknown languages (daily)
Generazione feed per Unknown languages (weekly)...
Feed salvato: feeds/unknown_languages_weekly.xml
Feed salv